In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

C:\Users\lamia\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
path = r"C:\Users\lamia\Downloads\dataset3-1 (1).csv"
df = pd.read_csv(path)

### Check Completeness per Cow

In [ ]:
# Group by cow and count unique dates
cow_date_counts = df.groupby('cow')['date'].nunique()

# Compare against expected number of days
expected_days = len(all_dates)
missing_days_per_cow = expected_days - cow_date_counts

print("\nMissing days per cow:")
print(missing_days_per_cow[missing_days_per_cow > 0])


Missing days per cow:
cow
1565     20
1797      1
2284     32
2399     27
2581      2
4372      2
5098      1
5394      8
5896     29
9994     27
10127     2
10567    13
Name: date, dtype: int64


### The number of hourly entries per cow per day

In [ ]:
hourly_counts = df.groupby(['cow', 'date']).size().reset_index(name='hourly_records')
print(hourly_counts)

        cow       date  hourly_records
0      1565 2013-10-01              24
1      1565 2013-10-02              24
2      1565 2013-10-03              24
3      1565 2013-10-04              24
4      1565 2013-10-05              24
...     ...        ...             ...
1091  10567 2013-10-27              24
1092  10567 2013-10-28              24
1093  10567 2013-10-29              24
1094  10567 2013-10-30              24
1095  10567 2013-10-31              13

[1096 rows x 3 columns]


### Filter for days with exactly 24 hours records

In [ ]:
full_days = hourly_counts[hourly_counts['hourly_records'] == 24]
print(f"\nDays with full days of records: {len(full_days)}")


Days with full days of records: 1078


### The number of full 24-hour days per cow

In [ ]:
full_days_per_cow = full_days.groupby('cow').size().reset_index(name='full_24h_days')
print("\nFull 24h days per cow:", full_days_per_cow)


Full 24h days per cow:       cow  full_24h_days
0    1565             21
1    1797             41
2    1919             40
3    2284             10
4    2340             42
5    2395             42
6    2399             15
7    2576             42
8    2581             40
9    2699             42
10   4102             40
11   4262             40
12   4279             41
13   4372             38
14   4495             42
15   4716             42
16   5098             40
17   5104             41
18   5128             40
19   5394             32
20   5541             42
21   5810             42
22   5896             12
23   8595             42
24   9481             42
25   9502             42
26   9601             42
27   9994             15
28  10127             40
29  10567             28


In [ ]:
print(f"Percentage of complete days: {len(full_days)/len(hourly_counts)*100:.2f}%")

Percentage of complete days: 98.36%


### Count how many 24h samples have less than 12 observations

In [ ]:
less_than_12_obs = hourly_counts[hourly_counts['hourly_records'] < 12]
print(f"Number of cow-day combinations with less than 12 hourly records: {len(less_than_12_obs)}")

if len(less_than_12_obs) > 0:
    print(less_than_12_obs)



Number of cow-day combinations with less than 12 hourly records: 1
      cow       date  hourly_records
843  5896 2013-10-13               2


# We will execute this code on the dataset that has 24h with less than 12 observations

In [ ]:
df_filtered = df.copy()
if len(less_than_12_obs) > 0:
    #Filter for valid cow-date combinations (at least 12 hourly records)
    valid_days = hourly_counts[hourly_counts['hourly_records'] >= 12]

    #Merge back to original data to keep only valid records
    df_filtered = pd.merge(df, valid_days[['cow', 'date']], on=['cow', 'date'], how='inner')

    # df_filtered now excludes cow-date combos with <12 hours
    print(df_filtered)

         cow       date  hour  IN_ALLEYS      REST      EAT  ACTIVITY_LEVEL  \
0      10127 2013-10-01     1      0.000  3600.000    0.000      -828.00000   
1      10127 2013-10-01     2   2931.783   444.528  223.689       460.79322   
2      10127 2013-10-01     3    257.740  2930.966  411.294      -460.14030   
3      10127 2013-10-01     4     63.220  3536.780    0.000      -803.34420   
4      10127 2013-10-01     5      0.000  3600.000    0.000      -828.00000   
...      ...        ...   ...        ...       ...      ...             ...   
26218   9994 2013-11-11    20   2439.672   617.248  543.079       476.47366   
26219   9994 2013-11-11    21   1920.242  1432.293  247.464        81.74621   
26220   9994 2013-11-11    22   2084.508  1213.487  302.004       181.26095   
26221   9994 2013-11-11    23    643.668  2746.255  210.076      -440.41985   
26222   9994 2013-11-11    24    553.511  3046.488    0.000      -612.13048   

       oestrus  calving  lameness  mastitis  LPS  a

**Perform cleaning by keeping only records with more that 18 samples**


In [ ]:
df_cleaned =df.copy()
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])

hourly_counts = df_cleaned.groupby(['cow', 'date']).size().reset_index(name='hour_count')


In [ ]:
# Identify 24h records by observation count

records_more_than_18_obs = hourly_counts[hourly_counts['hour_count'] > 18]
records_12_to_18_obs = hourly_counts[(hourly_counts['hour_count'] >= 12) & (hourly_counts['hour_count'] < 18)]
records_less_than_12_obs = hourly_counts[hourly_counts['hour_count'] < 12]

# Merge to filter full rows from original dataframe
df_more_than_18_obs = df_cleaned.merge(records_more_than_18_obs[['cow', 'date']], on=['cow', 'date'])
df_12_to_18_obs = df_cleaned.merge(records_12_to_18_obs[['cow', 'date']], on=['cow', 'date'])
df_less_than_12_obs = df_cleaned.merge(records_less_than_12_obs[['cow', 'date']], on=['cow', 'date'])

# Drop the records with less than 12 observations from the main dataset
df_filtered = df_cleaned[~df_cleaned.set_index(['cow', 'date']).index.isin(df_less_than_12_obs.set_index(['cow', 'date']).index)]

# Print dataset shapes
print("Original dataset shape:", df.shape)
print("After filtering (<12 obs removed):", df_filtered.shape)
print("Deleted rows (<12 obs):", df_less_than_12_obs.shape)
print("Filtered dataset (>18 obs):", df_more_than_18_obs.shape)
print("Filtered dataset (12–18 obs):", df_12_to_18_obs.shape)

In [ ]:
# Save datasets
df_more_than_18_obs.to_csv(r"filtered_dataset_more_than_18_obs.csv", index=False)

**Perform another cleaning by keeping only physiological classes**


Retain: mastitis, lameness, oestrus, calving, other_diseases, OK.


Remove: management changes, mixing, disturbance, accidents, LPS, acidosis.

In [ ]:
import pandas as pd

# Step 0: Load the dataset
file_path = r"C:\Users\lamia\Downloads\labelled&aligned_dataset3.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower()

# Step 1: Define useful and unwanted classes
useful_classes = ['mastitis', 'lameness', 'oestrus', 'calving', 'other_disease', 'ok']
unwanted_classes = ['management_changes', 'mixing', 'disturbance', 'accidents', 'lps', 'acidosis']

# Step 2: Drop unwanted classes
df_cleaned = df.drop(columns=unwanted_classes, errors='ignore')

# Step 3: Save the cleaned dataset
df_cleaned.to_csv('c:/users/lamia/Downloads/dataset3_aligned_cleaned_keep_physiological.csv', index=False)

# Optional: Show a sample
print(df_cleaned.head())
